In [1]:
!pip install -q transformers accelerate bitsandbytes beautifulsoup4 requests auto-gptq optimum

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 37.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 429.3/429.3 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig

model_name = "NousResearch/Nous-Hermes-2-Mistral-7B-DPO"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16"
)

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=bnb_config,
    torch_dtype="auto"
)

generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    temperature=0.7,
    do_sample=True
)

tokenizer_config.json:   0%|          | 0.00/1.65k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

...

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/120 [00:00<?, ?B/s]

Device set to use cuda:0


In [3]:
import requests

def get_steam_reviews(app_id, num_reviews=100):
    url = f"https://store.steampowered.com/appreviews/{app_id}"
    params = {
        "json": 1,
        "filter": "recent",
        "language": "english",
        "day_range": 90,
        "num_per_page": 100,
        "purchase_type": "all",
    }

    reviews = []
    cursor = "*"
    while len(reviews) < num_reviews:
        params["cursor"] = cursor
        response = requests.get(url, params=params)
        if not response.ok:
            break
        data = response.json()
        if not data.get("success"):
            break

        new_reviews = [r["review"] for r in data.get("reviews", [])]
        if not new_reviews:
            break

        reviews.extend(new_reviews)
        cursor = data.get("cursor", "")
        if not cursor:
            break

    return reviews[:num_reviews]


In [4]:
def chunk_reviews(reviews, max_chars=1500):
    chunks = []
    chunk = ""
    for review in reviews:
        if len(chunk) + len(review) < max_chars:
            chunk += review + "\n"
        else:
            chunks.append(chunk.strip())
            chunk = review + "\n"
    if chunk:
        chunks.append(chunk.strip())
    return chunks

In [5]:
def analyze_chunk(text):
    prompt = f"""
You are a helpful game reviewer. Analyze the following player reviews of a game. Summarize them in a short paragraph, then give 3 main pros and 3 cons. Be objective.

Reviews:
{text}

Output format:
Summary:
...
Pros:
- ...
- ...
- ...
Cons:
- ...
- ...
- ...
"""
    out = generator(prompt, max_new_tokens=400, do_sample=True, temperature=0.7)[0]['generated_text']
    return out.split("Output format:")[-1].strip()


In [6]:
def aggregate_summaries(all_chunk_results):
    cleaned_summaries = []
    for i, text in enumerate(all_chunk_results, 1):
        cleaned = text.strip()
        if "Summary:" in cleaned:
            cleaned_summaries.append(f"Chunk {i}:\n{cleaned}")

    merged_text = "\n\n".join(cleaned_summaries)

    prompt = f"""
You are a professional game critic. Based on the following chunk summaries from player reviews of a Steam game, write a final markdown-formatted review.

Instructions:
- Do NOT copy summaries directly. Summarize them into a unique overview.
- Be objective.
- Group and merge similar ideas.
- Focus only on player reviews — do not make up external references.
- Use clean markdown structure as below:

### Summary
(2–3 sentence summary of the game)

### Pros
- ...
- ...
- ...

### Cons
- ...
- ...
- ...

### Final Recommendation
Yes or No — and one sentence why.

Summaries:
{merged_text}

### Review:
"""

    result = generator(prompt, max_new_tokens=512, temperature=0.7, do_sample=True)[0]['generated_text']
    return result.split("### Review:")[-1].strip()


In [8]:
import re

url = input("Enter steam game link: ")

#url = "]"
match = re.search(r'/app/(\d+)/', url)
app_id = match.group(1) if match else None

print("Fetching reviews from Steam...")
reviews = get_steam_reviews(app_id, num_reviews=100)
chunks = chunk_reviews(reviews, max_chars=1500)

chunk_results = []
for i, chunk in enumerate(chunks):
    print(f"Processing chunk {i+1}/{len(chunks)}")
    chunk_summary = analyze_chunk(chunk)
    chunk_results.append(chunk_summary)

print("Creating final structured analysis...")
final_result = aggregate_summaries(chunk_results)

from IPython.display import Markdown
display(Markdown("## 🎮 Steam Game Analysis\n\n" + final_result))


Enter steam game link: https://store.steampowered.com/app/1091500/Cyberpunk_2077/
Fetching reviews from Steam...
Processing chunk 1/13
Processing chunk 2/13
Processing chunk 3/13
Processing chunk 4/13
Processing chunk 5/13
Processing chunk 6/13
Processing chunk 7/13
Processing chunk 8/13
Processing chunk 9/13


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Processing chunk 10/13
Processing chunk 11/13
Processing chunk 12/13
Processing chunk 13/13
Creating final structured analysis...


## 🎮 Steam Game Analysis

Based on the reviews, the game Cyberpunk 2077 is generally appreciated for its immersive world, engaging gameplay, and captivating storylines. Many players find the game to be a fun and worthwhile experience. However, there are also criticisms regarding technical issues (glitches, bugs), unimpressive fighting mechanics, repetitive gameplay, and a messy launch. The game has received both positive and negative reviews, with some players highly recommending it and others expressing disappointment.

Pros:
- Immersive world
- Engaging gameplay
- Captivating storylines

Cons:
- Technical issues (glitches, bugs)
- Unimpressive fighting mechanics
- Repetitive gameplay
- Messy launch

Final Recommendation: Yes, with reservations.